# Verify `original_rawboost.SSI_additive_noise` == `new_rawboost.SSINoiseAugment`

In [47]:
import importlib
import numpy as np

# Load modules directly from src/ to avoid the src/logging.py shadow
import importlib.util

def _load_from_src(module_name):
    spec = importlib.util.spec_from_file_location(module_name, f"src/{module_name}.py")
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

In [48]:
_orig = _load_from_src("original_rawboost")
_new = _load_from_src("new_rawboost")

SSI_additive_noise = _orig.SSI_additive_noise
SSINoiseAugment = _new.SSINoiseAugment
SSINoiseConfig = _new.SSINoiseConfig

## Shared parameters and test signal

In [49]:
SEED = 42
SAMPLE_RATE = 16000

# Parameters for the original API (flat kwargs) — matches argparse defaults
orig_params = dict(
    SNRmin=10, SNRmax=40,
    nBands=5, minF=20, maxF=8000,
    minBW=100, maxBW=1000,
    minCoeff=10, maxCoeff=100,
    minG=0, maxG=0,
    fs=SAMPLE_RATE,
)

# Equivalent config for the rewritten API — matches SSINoiseConfig defaults
new_cfg = SSINoiseConfig(
    snr_min=10, snr_max=40,
    num_bands=5, freq_lo=20, freq_hi=8000,
    bw_lo=100, bw_hi=1000,
    order_lo=10, order_hi=100,
    gain_lo=0, gain_hi=0,
    sample_rate=SAMPLE_RATE,
)

np.random.seed(0)
audio = np.random.randn(SAMPLE_RATE)  # 1 second of random audio

## Run original

In [50]:
np.random.seed(SEED)
result_original = SSI_additive_noise(audio.copy(), **orig_params)
print("Original  - first 10 values:", result_original[:10])

Original  - first 10 values: [ 1.78916734  0.43413857  1.03683072  2.24399707  1.85928725 -0.92702155
  0.97992157 -0.18746541 -0.0774166   0.37388175]


## Run rewritten version

In [51]:
augment = SSINoiseAugment(new_cfg)

np.random.seed(SEED)
result_new = augment(audio.copy())
print("Rewritten - first 10 values:", result_new[:10])

Rewritten - first 10 values: [ 1.78916734  0.43413857  1.03683072  2.24399707  1.85928725 -0.92702155
  0.97992157 -0.18746541 -0.0774166   0.37388175]


## Verify equality

In [52]:
assert result_original.shape == result_new.shape, "Shape mismatch!"

max_diff = np.max(np.abs(result_original - result_new))
print(f"Max absolute difference: {max_diff}")

# The rewritten version uses FFT-based convolution (fftconvolve) instead of
# time-domain lfilter for speed. The results are mathematically identical but
# floating-point rounding differs slightly, so we use a tight tolerance.
np.testing.assert_allclose(result_original, result_new, atol=1e-10, rtol=1e-10)
print("PASSED: outputs match within numerical tolerance (atol=1e-10, rtol=1e-10).")

Max absolute difference: 0.0
PASSED: outputs match within numerical tolerance (atol=1e-10, rtol=1e-10).


## Profiling

In [53]:
import cProfile
import pstats
from pathlib import Path

PROFILE_DIR = Path("profiles")
PROFILE_DIR.mkdir(exist_ok=True)

N_CALLS = 100
segment = np.random.randn(64600).astype(np.float32)  # realistic segment length

### Profile original

In [54]:
def run_original():
    for _ in range(N_CALLS):
        SSI_additive_noise(segment.copy(), **orig_params)

prof_orig = cProfile.Profile()
prof_orig.runcall(run_original)
prof_orig.dump_stats(str(PROFILE_DIR / "original.prof"))

print(f"=== Original ({N_CALLS} calls) ===")
pstats.Stats(prof_orig).strip_dirs().sort_stats("cumulative").print_stats(15)

=== Original (100 calls) ===
         131600 function calls (131491 primitive calls) in 6.238 seconds

   Ordered by: cumulative time
   List reduced from 294 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.001    0.000    6.228    6.228 3076890840.py:1(run_original)
  101/100    0.276    0.003    6.133    0.061 original_rawboost.py:111(SSI_additive_noise)
      200    0.029    0.000    4.779    0.024 _linalg.py:2623(norm)
      200    4.750    0.024    4.750    0.024 {method 'dot' of 'numpy.ndarray' objects}
      100    0.001    0.000    0.769    0.008 original_rawboost.py:52(filterFIR)
      100    0.002    0.000    0.761    0.008 _signaltools.py:2043(lfilter)
      100    0.024    0.000    0.758    0.008 _shape_base_impl.py:281(apply_along_axis)
      600    0.001    0.000    0.728    0.001 numeric.py:806(convolve)
      600    0.726    0.001    0.726    0.001 {built-in method numpy._core._multiarray_umath.correl

### Profile rewritten

In [55]:
def run_new():
    for _ in range(N_CALLS):
        augment(segment.copy())

prof_new = cProfile.Profile()
prof_new.runcall(run_new)
prof_new.dump_stats(str(PROFILE_DIR / "new.prof"))

print(f"=== Rewritten ({N_CALLS} calls) ===")
pstats.Stats(prof_new).strip_dirs().sort_stats("cumulative").print_stats(15)

=== Rewritten (100 calls) ===
         130972 function calls (130870 primitive calls) in 1.494 seconds

   Ordered by: cumulative time
   List reduced from 224 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.368    0.004    1.496    0.015 new_rawboost.py:61(__call__)
        1    0.001    0.001    1.306    1.306 273799424.py:1(run_new)
      100    0.021    0.000    0.713    0.007 new_rawboost.py:143(_apply_fir)
      100    0.001    0.000    0.687    0.007 _signaltools.py:2043(lfilter)
      100    0.003    0.000    0.684    0.007 _shape_base_impl.py:281(apply_along_axis)
      600    0.001    0.000    0.684    0.001 numeric.py:806(convolve)
      600    0.673    0.001    0.673    0.001 {built-in method numpy._core._multiarray_umath.correlate}
      100    0.000    0.000    0.660    0.007 _signaltools.py:2227(<lambda>)
      100    0.026    0.000    0.405    0.004 new_rawboost.py:80(_build_composite_fir)
      500  

### Visualize with snakeviz

Run either of these in a terminal to open the interactive flame chart:
```
snakeviz profiles/original.prof
snakeviz profiles/new.prof
```